In [ ]:
import couchdb
import random
import pandas as pd
import numpy as np
from scipy.special import softmax

In [2]:
user = "admin"
password = "admin"
couchserver = couchdb.Server("http://%s:%s@172.26.134.187:5984/" % (user, password))

In [3]:
db = couchserver['twitter']
rows = db.view('_design/CrimeInfo/_view/TweetData')

In [4]:
data = [(row['id'], row['value']) for row in rows]
#random.seed(0)
#random.shuffle(data)

In [5]:
numTweets = 100

In [6]:
data = pd.DataFrame(data).rename({0: "id", 1: "tweet"}, axis='columns')[:numTweets]

In [7]:
dfSentiment = data.copy(deep=True)
dfSentiment

id                                              tweet
0   1520266206548213760  When you’re stuck underneath a pup  \n\n#Albus...
1   1520342297728241664  Just posted a photo @ Organ Pipes National Par...
2    493802764774825984                                       Finally home
3    493802997793570817     When Harry met Anna 😉😍💕 http://t.co/8cnpbUlnVz
4    493803077489537024  Home Weather - Temp: 12.4 °C. Wind: 8.9 km/h N...
..                  ...                                                ...
95   493921747066822656  Chris Lannon: number of products containing pu...
96   493921971080396801  Morning Flight (at Essendon Airport (MEB)) — h...
97   493921984066387969  On a carriage with a whole lot of teens on som...
98   493921998393724928                    @Miff6Lorraine @Cameron_Lincoln
99   493921999861739523  @MolksTVTalk if we are going to claim it like ...

[100 rows x 2 columns]

In [8]:
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer

# remove hashtags
def hashtags(text):
  hash = re.findall(r"#(\w+)", text)
  return hash

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/m.rossi/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [9]:
from emot.emo_unicode import UNICODE_EMOJI, EMOTICONS_EMO
# translate emoji
def emoji(text):
  for emot in UNICODE_EMOJI:
    if text == None:
      text = text
    else:
      text = text.replace(emot, "_".join(UNICODE_EMOJI[emot].replace(",", "").replace(":", "").split()))
    return text

In [10]:
# remove retweet username and tweeted at @username
def remove_users(tweet):
  '''Takes a string and removes retweet and @user information'''
  tweet = re.sub('(@[A-Za-z]+[A-Za-z0-9-_]+)', '', tweet) 
  # remove tweeted at
  return tweet

In [11]:
# remove links
def remove_links(tweet):
  '''Takes a string and removes web links from it'''
  tweet = re.sub(r'http\S+', '', tweet) # remove http links
  tweet = re.sub(r'bit.ly/\S+', '', tweet) # remove bitly links
  tweet = tweet.strip('[link]') # remove [links]
  return tweet
def clean_html(text):
  html = re.compile('<.*?>')#regex
  return html.sub(r'',text)

In [12]:
# remove non ascii character
def non_ascii(s):
  return "".join(i for i in s if ord(i)<128)

def lower(text):
  return text.lower()

In [13]:
# remove email address
def email_address(text):
  email = re.compile(r'[\w\.-]+@[\w\.-]+')
  return email.sub(r'',text)

def punct(text):
  token=RegexpTokenizer(r'\w+')#regex
  text = token.tokenize(text)
  text= " ".join(text)
  return text 

In [14]:
# remove stopwords
def removeStopWords(str):
#select english stopwords
  cachedStopWords = set(stopwords.words("english"))
#add custom words
  cachedStopWords.update(('and','I','A','http','And','So','arnt','This','When','It','many','Many','so','cant','Yes','yes','No','no','These','these','mailto','regards','ayanna','like','email'))
#remove stop words
  new_str = ' '.join([word for word in str.split() if word not in cachedStopWords]) 
  return new_str

In [15]:
# special characters removal
def remove_(tweet):
  tweet = re.sub('([_]+)', "", tweet)
  return tweet

In [16]:
#apply all the functions above
#dfSentiment['hashtag'] = dfSentiment.tweet.apply(func = hashtags)
dfSentiment['new_tweet'] = dfSentiment.tweet.apply(func = emoji)
dfSentiment['new_tweet'] = dfSentiment.new_tweet.apply(func = remove_users)
dfSentiment['new_tweet'] = dfSentiment.new_tweet.apply(func = clean_html)
dfSentiment['new_tweet'] = dfSentiment.new_tweet.apply(func = remove_links)
dfSentiment['new_tweet'] = dfSentiment.new_tweet.apply(func = non_ascii)
dfSentiment['new_tweet'] = dfSentiment.new_tweet.apply(func = lower)
dfSentiment['new_tweet'] = dfSentiment.new_tweet.apply(func = email_address)
dfSentiment['new_tweet'] = dfSentiment.new_tweet.apply(func = removeStopWords)
dfSentiment['new_tweet'] = dfSentiment.new_tweet.apply(func = clean_html)
dfSentiment['new_tweet'] = dfSentiment.new_tweet.apply(func = punct)
dfSentiment['new_tweet'] = dfSentiment.new_tweet.apply(func = remove_)

In [17]:
dfSentiment

id                                              tweet  \
0   1520266206548213760  When you’re stuck underneath a pup  \n\n#Albus...   
1   1520342297728241664  Just posted a photo @ Organ Pipes National Par...   
2    493802764774825984                                       Finally home   
3    493802997793570817     When Harry met Anna 😉😍💕 http://t.co/8cnpbUlnVz   
4    493803077489537024  Home Weather - Temp: 12.4 °C. Wind: 8.9 km/h N...   
..                  ...                                                ...   
95   493921747066822656  Chris Lannon: number of products containing pu...   
96   493921971080396801  Morning Flight (at Essendon Airport (MEB)) — h...   
97   493921984066387969  On a carriage with a whole lot of teens on som...   
98   493921998393724928                    @Miff6Lorraine @Cameron_Lincoln   
99   493921999861739523  @MolksTVTalk if we are going to claim it like ...   

                                            new_tweet  
0   youre stuck underneath pup albus albiedog pupp...  
1              posted photo organ pipes national park  
2                                        finally home  
3                                      harry met anna  
4   home weather temp 12 4 c wind 8 9 km h nne bar...  
..                                                ...  
95  chris lannon number products containing pulses...  
96             morning flight at essendon airport meb  
97  carriage whole lot teens excursion mckinnon st...  
98                                                     
99  going claim give us ratings results secondary ...  

[100 rows x 3 columns]

In [18]:
from transformers import AutoModelForSequenceClassification
from transformers import TFAutoModelForSequenceClassification
from transformers import AutoTokenizer, AutoConfig

MODEL = f"cardiffnlp/twitter-roberta-base-sentiment-latest"
tokenizer = AutoTokenizer.from_pretrained(MODEL)
config = AutoConfig.from_pretrained(MODEL)
model = AutoModelForSequenceClassification.from_pretrained(MODEL)

Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [19]:
def sentiment_score(review):
    tokens = tokenizer.encode(review, return_tensors='pt')
    result = model(tokens)
    scores = result[0][0].detach().numpy()
    scores = softmax(scores)
    ranking = np.argsort(scores)
    ranking = ranking[::-1]
    #for i in range(scores.shape[0]):
        #l = config.id2label[ranking[i]]
        #s = scores[ranking[i]]
        #print(f"{i+1}) {l} {np.round(float(s), 4)}")
    
    return (config.id2label[ranking[0]], scores[ranking[0]])

dfSentiment['sentiment'] = dfSentiment['new_tweet'].apply(lambda x: sentiment_score(x[:512]))

In [20]:
dfSentiment

id                                              tweet  \
0   1520266206548213760  When you’re stuck underneath a pup  \n\n#Albus...   
1   1520342297728241664  Just posted a photo @ Organ Pipes National Par...   
2    493802764774825984                                       Finally home   
3    493802997793570817     When Harry met Anna 😉😍💕 http://t.co/8cnpbUlnVz   
4    493803077489537024  Home Weather - Temp: 12.4 °C. Wind: 8.9 km/h N...   
..                  ...                                                ...   
95   493921747066822656  Chris Lannon: number of products containing pu...   
96   493921971080396801  Morning Flight (at Essendon Airport (MEB)) — h...   
97   493921984066387969  On a carriage with a whole lot of teens on som...   
98   493921998393724928                    @Miff6Lorraine @Cameron_Lincoln   
99   493921999861739523  @MolksTVTalk if we are going to claim it like ...   

                                            new_tweet               sentiment  
0   youre stuck underneath pup albus albiedog pupp...   (Neutral, 0.82787895)  
1              posted photo organ pipes national park   (Neutral, 0.73221326)  
2                                        finally home  (Positive, 0.86881274)  
3                                      harry met anna    (Neutral, 0.8515799)  
4   home weather temp 12 4 c wind 8 9 km h nne bar...   (Neutral, 0.85651684)  
..                                                ...                     ...  
95  chris lannon number products containing pulses...    (Neutral, 0.8454664)  
96             morning flight at essendon airport meb   (Neutral, 0.82303685)  
97  carriage whole lot teens excursion mckinnon st...   (Neutral, 0.90124226)  
98                                                      (Positive, 0.4012986)  
99  going claim give us ratings results secondary ...    (Neutral, 0.8915859)  

[100 rows x 4 columns]